In [5]:
from collections import Counter,defaultdict
from bs4 import BeautifulSoup
import os
import glob
import nltk
import math
from __future__ import division

In [6]:
import urllib2
from bs4 import BeautifulSoup as bs
import re

"""
test python file for parsing erowid
add doc strings
"""

#parent url
erowid_url = "https://erowid.org/experiences/"

#helper functions:

def read_links(start_page):
    """
    takes a url and returns the links on that page
    """
    response = urllib2.urlopen(start_page)
    #convert to soup in order to strain out the anchors and the urls
    soup = bs(response.read(), "lxml")
    href_tags = [a['href'] for a in soup.findAll('a') if a.has_key('href')]
    #return links from page -> these are the links to the drug experinces
    #use regex here later?
    return href_tags

def convert_local_to_absolute_url(parent_url, url_list):
    """
    takes a list of relative urls and converts to absolute urls
    can be simpliefied later
    """
    full_url_list = []
    for l in url_list:
        #append the parent url to the relative url 
        full_url = parent_url + l
        #make a new list of absolute urls
        full_url_list.append(full_url)
    return full_url_list
    


In [ ]:
#changed the doc string in the url to retrieve all links to the experiences on one page
#(Start=0&Max=24810) the 24810 is the amount of current experiences
#LATER-> get the Max parameter from the webpage + append into query parameter so is constantly 
#retrieving All Experiences
all_experience = "https://erowid.org/experiences/exp.cgi?ShowViews=0&Cellar=0&Start=0&Max=24810"

#regular expression to match experiences urls. The [0-9] matches the 5 number experience id 
#e.g will match exp.php?ID=107831
experience_regex = '[exp.php?ID=]*[0-9]{5}'
  
def get_experiences(url, regex):
    """
    takes a url and a regex returns a list of all the urls satistfying the regex on the page
    """
    experience = []
    #we find all the urls on the input url using previously defined function read_links
    for l in read_links(url):
        #if the urls on the page match the previously defined regex, return them
        match = re.match(regex, l)
        if match:
            experience.append(l)
        else:
            pass
    return experience
        
#we save all the relative urls pointing to each drug experience    
relative_urls =  get_experiences(all_experience, experience_regex)

#the absolute urls for each experience is:
absolute_urls = (convert_local_to_absolute_url(erowid_url, relative_urls))

def retrieve_drug_and_gender(url_list):
    """
    takes a list of the urls and opens them
    then finds the foottable and extracts the gender
    then finds the beginning table and extracts the drug   
    """
    drug_list = []
    for l in url_list:
        #open each url
        #write a 404 check
        try:
            response = urllib2.urlopen(l)
        except:
            pass
        else:
            #convert to soup
            soup = bs(response.read(), "lxml")
            #find the html table with the class footdata
            # write a check for the table
            if (soup.find(class_='footdata')):
                end_table = soup.find(class_='footdata')
                rows = end_table.findAll('tr')
            else:
                pass
            #the gender is listed in the table data text as the 3rd element in the table
            gender_in_table = [[td.text for td in tr.findAll("td")] for tr in rows]
            gender = gender_in_table[1]
            #get the table containing the drug information
            drug_class = [td.find('a') for td in soup.findAll('td', {'class' : 'dosechart-substance'})]
            drug = [drug.text for drug in drug_class if drug]
            drug_gender = (drug, gender)
            #need to return a list of tuples
            drug_list.append(drug_gender)
    return drug_list


#a list of genders and the drugs
data_set = retrieve_drug_and_gender(absolute_urls)



In [ ]:
def count_gender(drug_gender_list):
    """
    takes output of retrieve_drug_and_gender and 
    returns a dict of theccount of each gender for each drug
    """
    male_dict = {}
    female_dict = {}
    #select the second list in the item -> this is the part 
    #containing the information about gender
    for i in drug_gender_list:
        for gender in i[1]:
            #if gender is male, male_count is + 1
            if "Male" in gender:
                # we need to add 1 to the count specific to that drug
                #use a set to ensure no repetition of drug name in table
                for drug in set(i[0]):
                    if not drug in male_dict:
                        #if the drug is not already a key in the dict, then the count is 1
                        # as we havent seen it before
                        male_dict[drug] = 1
                    else:
                        # the count is incremented by 1
                        male_dict[drug] += 1
            #add 1 to female count if the gender is female
            if "Female" in gender:
                #check that there is no repetition of the same drug in a single entry
                for drug in set(i[0]):
                    if not drug in female_dict:
                        #the count is 1
                        female_dict[drug] = 1 
                    else:
                        # the count is incremented by 1
                        female_dict[drug] += 1
    return female_dict, male_dict

#returns a tuple with two dicts. the first dict is the count of the female drug users, the second dict
# is a count of the male drug users
complete_data_dict = count_gender(data_set)

#print complete_data_dict


In [ ]:
#display the results as bar plots

%matplotlib inline
import matplotlib.pyplot as plt

#get the output dict from the counted gender
dicts = count_gender(data_set)

#the first dict is female drug users
female_drugs = dicts[0]
#the second dict is male drug users
male_drugs = dicts[1]

#get the top 5 used drugs from male and female users
top_female_drugs = Counter(female_drugs).most_common(5)
top_male_drugs = Counter(male_drugs).most_common(5)

#display the results as a bar chart
#female drug users plotted onto a graph
plt.bar(range(len(top_female_drugs)), [x[1] for x in top_female_drugs], align="center")
plt.xticks(range(len(top_female_drugs)), [x[0] for x in top_female_drugs])
#label the axis
plt.ylabel('amount of users')
plt.title('Female drug users')

plt.show()

#male drug users plotted onto a drug
plt.bar(range(len(top_male_drugs)), [y[1] for y in top_male_drugs], align="center", rotation='vertical')
plt.xticks(range(len(top_male_drugs)), [y[0] for y in top_male_drugs])
#label the axis 
plt.ylabel('amount of users')
plt.title('Male drug users')

plt.show()

